# Load model
Our best model is a TrOCR model we uploaded to huggingface. It has a Spanish language base model and is fine-tuned on the hackathon dataset. (Read more in our report)

In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("MarieRoald/DeepLearn24OCRChallenge").to(device)
model.generation_config.max_length = 128
model.generation_config.early_stopping = False

processor = TrOCRProcessor.from_pretrained("MarieRoald/DeepLearn24OCRChallenge")

config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

2024-07-26 16:32:31.686677: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 16:32:31.718947: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-26 16:32:31.892587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 16:32:31.892740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 16:32:31.914537: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

# Load test data
Loading our line-level data from our github repo

In [2]:
# !git clone https://github.com/MarieRoald/deeplearn24-NLPRenaissanceChallenge.git

In [3]:
import pandas as pd

metadata_df = pd.read_csv("../data/2_bounding_box/Doc-UFCN_processed/metadata.csv")

test_df = metadata_df[metadata_df.split == "test"]
test_df = test_df.sort_values(["spread_number", "spread_side", "line_number"])
test_df.index = range(len(test_df))
test_df

,file_name,spread_number,spread_side,split,line_number,transcription,x,y,w,h
0,test/image_14_left_001.png,14,left,test,1,NaN,88,69,389,39
1,test/image_14_left_002.png,14,left,test,2,NaN,87,98,390,40
2,test/image_14_left_003.png,14,left,test,3,NaN,86,127,389,39
3,test/image_14_left_004.png,14,left,test,4,NaN,85,158,392,37
4,test/image_14_left_005.png,14,left,test,5,NaN,86,187,389,39
...,...,...,...,...,...,...,...,...,...,...
133,test/image_16_right_043.png,16,right,test,43,NaN,21,603,389,33
134,test/image_16_right_044.png,16,right,test,44,NaN,22,637,389,28
135,test/image_16_right_045.png,16,right,test,45,NaN,21,663,380,32
136,test/image_16_right_046.png,16,right,test,46,NaN,23,691,350,34


In [4]:
from pathlib import Path
from tqdm import trange
from PIL import Image

test_set_path = Path("../data/2_bounding_box/Doc-UFCN_processed/")
test_images = test_df.file_name.apply(lambda file_name: test_set_path / file_name).apply(Image.open).to_list()

pixel_values = processor(test_images, return_tensors="pt")["pixel_values"].to(device)
estimated_tokens = [model.generate(pixel_values[i].unsqueeze(0)) for i in trange(len(test_images))]
test_df["prediction"] = [
    processor.batch_decode(estimated_tokens[i], skip_special_tokens=True)[0]
    for i in trange(len(test_images))
]

100%|██████████| 138/138 [00:00<00:00, 6688.94it/s]


In [6]:
test_output = Path("../data/best_model_test_predictions/")
test_output.mkdir(exist_ok=True, parents=True)

test_df.to_csv(test_output / "predictions.csv", index=False)

# Concat model outputs to page data

In [8]:

handout_data_page_start = 26
current_page = handout_data_page_start

for spread, df_ in test_df.groupby("spread_number"):
    assert df_["spread_side"].to_list()[0] == "left" and df_["spread_side"].to_list()[-1] == "right"
    for side, df__ in df_.groupby("spread_side"):
        page_text = "\n".join(df__.prediction)
        with open(test_output / f"image_{current_page}_prediction.txt", "w+") as f:
            f.write(page_text)
        current_page += 1
